# Flutter Localization Synchronizer

This notebook automates the synchronization of variables between `app_localizations_en.dart` and `app_localizations.dart` in a Flutter project. It identifies variables present in the English localization file but missing in the main localization file, and adds them automatically.

## Import Required Libraries

Import necessary libraries such as os and re for file handling and regex operations.

In [ ]:
import os
import re
import sys
from pathlib import Path

## Load Localization Files

Read the contents of app_localizations_en.dart and app_localizations.dart into memory.

In [ ]:
# Define path to localization files
# Update these paths to match your project structure
lib_dir = Path('c:/Users/wailik/Documents/Code/Flutter/demo/demo/lib')
gen_dir = lib_dir / 'generated'
l10n_dir = lib_dir / 'l10n'

en_file_path = l10n_dir / 'app_localizations_en.dart'
main_file_path = l10n_dir / 'app_localizations.dart'

# Function to read file content
def read_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

# Read the localization files
en_content = read_file(en_file_path)
main_content = read_file(main_file_path)

if en_content is None or main_content is None:
    print("Cannot proceed without both files.")
else:
    print(f"Successfully loaded both localization files.")
    print(f"English file size: {len(en_content)} characters")
    print(f"Main file size: {len(main_content)} characters")

## Extract Variables from app_localizations_en.dart

Use regex to extract all variable names from app_localizations_en.dart. We'll look for patterns that match Flutter localization class variables.

In [ ]:
def extract_variables(content):
    """Extract localization variable names from the content"""
    # This regex pattern looks for methods with String return type and getters
    # Pattern explanation:
    # 1. Find String methods: "String get variableName =>"
    # 2. Find String methods with parameters: "String variableName(params) =>"
    
    # Pattern for getters (String get varName =>)
    getter_pattern = r'String\s+get\s+(\w+)\s*=>'
    
    # Pattern for method declarations (String varName() {)
    method_pattern = r'String\s+(\w+)\s*\([^)]*\)\s*{'
    
    getter_vars = re.findall(getter_pattern, content)
    method_vars = re.findall(method_pattern, content)
    
    # Combine all found variables
    all_vars = getter_vars + method_vars
    return all_vars

# Extract variables from the English localization file
if en_content:
    en_variables = extract_variables(en_content)
    print(f"Found {len(en_variables)} variables in the English localization file:")
    print(en_variables[:10], "..." if len(en_variables) > 10 else "")
else:
    en_variables = []

## Compare and Synchronize Variables

Compare the extracted variables with those in app_localizations.dart and identify missing variables.

In [ ]:
def find_missing_variables(en_vars, main_content):
    """Find variables that exist in en_vars but not in main_content"""
    missing_vars = []
    
    for var in en_vars:
        # Check for getter pattern
        getter_pattern = r'String\s+get\s+' + var + r'\s*=>'
        
        # Check for method pattern
        method_pattern = r'String\s+' + var + r'\s*\('
        
        if not re.search(getter_pattern, main_content) and not re.search(method_pattern, main_content):
            missing_vars.append(var)
    
    return missing_vars

# Find missing variables
if en_variables and main_content:
    missing_variables = find_missing_variables(en_variables, main_content)
    print(f"Found {len(missing_variables)} variables missing in the main localization file:")
    print(missing_variables)
else:
    missing_variables = []

## Extract Variable Definitions

For each missing variable, we need to extract the full definition from the English file to add it to the main file.

In [ ]:
def extract_variable_definition(content, var_name):
    """Extract the full definition of a variable from the content"""
    # Try to find getter definition
    getter_pattern = r'(String\s+get\s+' + var_name + r'\s*=>.*?;)'
    
    # Try to find method definition
    method_pattern = r'(String\s+' + var_name + r'\s*\([^)]*\)\s*{.*?})'
    
    # Search for both patterns
    getter_match = re.search(getter_pattern, content, re.DOTALL)
    method_match = re.search(method_pattern, content, re.DOTALL)
    
    if getter_match:
        return getter_match.group(1)
    elif method_match:
        return method_match.group(1)
    else:
        return None

# Extract the full definitions of missing variables
variable_definitions = {}
for var in missing_variables:
    definition = extract_variable_definition(en_content, var)
    if definition:
        variable_definitions[var] = definition
    else:
        print(f"Warning: Could not extract definition for variable '{var}'")

print(f"Successfully extracted {len(variable_definitions)} variable definitions")

## Write Updated app_localizations.dart

Add missing variables to app_localizations.dart and write the updated content back to the file.

In [ ]:
def insert_variables_into_main_file(main_content, variable_definitions):
    """Insert missing variables into the main localization file"""
    # Find the end of the class definition
    class_end_pattern = r'}'
    
    # Find the last occurrence of the class closing brace
    matches = list(re.finditer(class_end_pattern, main_content))
    if not matches:
        return main_content, False
    
    # Get the position of the last class closing brace
    last_brace_pos = matches[-1].start()
    
    # Insert the missing variables before the last closing brace
    variables_text = "\n\n  // Auto-added missing variables\n"
    for var, definition in variable_definitions.items():
        variables_text += f"  {definition}\n"
    
    # Insert the new text
    updated_content = main_content[:last_brace_pos] + variables_text + main_content[last_brace_pos:]
    
    return updated_content, True

# Create the updated main file content
if variable_definitions:
    updated_main_content, success = insert_variables_into_main_file(main_content, variable_definitions)
    if success:
        print(f"Successfully created updated content with {len(variable_definitions)} new variables")
    else:
        print("Failed to update the main localization file")
else:
    updated_main_content = main_content
    print("No variables to add to the main localization file")

## Preview Changes

Preview the changes that will be made to the main localization file.

In [ ]:
def preview_changes(original, updated):
    """Show a diff-like preview of the changes"""
    if original == updated:
        print("No changes to preview")
        return
    
    # Calculate lines that differ
    original_lines = original.split('\n')
    updated_lines = updated.split('\n')
    
    # Find the first different line
    start_diff = None
    for i in range(min(len(original_lines), len(updated_lines))):
        if original_lines[i] != updated_lines[i]:
            start_diff = i
            break
    
    if start_diff is None:
        # Files differ only in length
        start_diff = min(len(original_lines), len(updated_lines))
    
    # Show context around the changes
    context_lines = 5
    start_line = max(0, start_diff - context_lines)
    
    print("Preview of changes:")
    print("-------------------")
    
    # Show a few lines before the changes for context
    if start_line > 0:
        print("...(lines omitted)...")
    
    for i in range(start_line, min(len(updated_lines), start_diff + 20)):
        prefix = "  "
        if i >= len(original_lines) or original_lines[i] != updated_lines[i]:
            prefix = "+ "  # added or modified line
        print(f"{prefix}{updated_lines[i]}")
    
    if len(updated_lines) > start_diff + 20:
        print("...(more lines)...")

# Preview the changes
if main_content != updated_main_content:
    preview_changes(main_content, updated_main_content)
else:
    print("No changes to preview")

## Save Updated File

Save the updated content back to app_localizations.dart, with an option to create a backup of the original file.

In [ ]:
def write_file(file_path, content, create_backup=True):
    """Write content to file with optional backup"""
    try:
        # Create backup if requested
        if create_backup and os.path.exists(file_path):
            backup_path = str(file_path) + '.bak'
            with open(file_path, 'r', encoding='utf-8') as original:
                with open(backup_path, 'w', encoding='utf-8') as backup:
                    backup.write(original.read())
            print(f"Created backup at {backup_path}")
        
        # Write updated content
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)
        return True
    except Exception as e:
        print(f"Error writing to {file_path}: {e}")
        return False

# Prompt for confirmation before writing
print("\nReady to update the main localization file.")
print(f"Number of variables to add: {len(variable_definitions)}")

# Uncomment below to actually write the file
# confirmation = input("Type 'yes' to confirm writing to the file: ")
# if confirmation.lower() == 'yes':
#     success = write_file(main_file_path, updated_main_content)
#     if success:
#         print(f"Successfully updated {main_file_path}")
#     else:
#         print("Failed to update the file")
# else:
#     print("Operation cancelled")

# For safety, just print a message instead of actually writing the file
print("\nTo actually save changes, uncomment the confirmation code block above.")
print("(File writing disabled by default for safety)")

## Summary

This notebook has:
1. Loaded both localization files
2. Extracted variables from the English localization file
3. Identified variables missing in the main localization file
4. Generated updated content with the missing variables
5. Provided a preview of the changes

To apply the changes, uncomment the confirmation block in the "Save Updated File" section.